# Non-linear recharge models
*R.A. Collenteur, University of Graz*

This notebook explains the use of the `RechargeModel` stress model to simulate the combined effect of precipitation and potential evaporation on the groundwater levels. For the computation of the groundwater recharge, four recharge models are currently available:

- `Linear` ([Berendrecht et al., 2003](#References); [von Asmuth et al., 2008](#References))
- `Berendrecht` ([Berendrecht et al., 2006](#References))
- `FlexModel` ([Collenteur et al., 2021](#References))
- `Peterson` ([Peterson & Western, 2014](#References))

The first model is a simple linear function of precipitation and potential evaporation while the latter two are simulate a non-linear response of recharge to precipitation using a soil-water balance concepts. Detailed descriptions of these models can be found in articles listed in the [References](#References) at the end of this notebook.



In [ ]:
import pandas as pd
import pastas as ps
import matplotlib.pyplot as plt

ps.show_versions()
ps.set_log_level("INFO")

## Read Input data
Input data handling is similar to other stressmodels. The only thing that is necessary to check is that the precipitation and evaporation are provided in mm/day. This is necessary because the parameters for the non-linear recharge models are defined in mm for the length unit and days for the time unit. It is possible to use other units, but this would require manually setting the initial values and parameter boundaries for the recharge models.

In [ ]:
head = pd.read_csv(
    "data/B32C0639001.csv", parse_dates=["date"], index_col="date"
).squeeze()

# Make this millimeters per day
evap = pd.read_csv("data/evap_260.csv", index_col=0, parse_dates=[0]).squeeze()
rain = pd.read_csv("data/rain_260.csv", index_col=0, parse_dates=[0]).squeeze()

ps.plots.series(
    head,
    [evap, rain],
    figsize=(10, 6),
    labels=["Head [m]", "Evap [mm/d]", "Rain [mm/d]"],
);

## Make a basic model
The normal workflow may be used to create and calibrate the model.
1. Create a Pastas `Model` instance
2. Choose a recharge model. All recharge models can be accessed through the recharge subpackage (`ps.rch`).
3. Create a `RechargeModel` object and add it to the model
4. Solve and visualize the model



In [ ]:
ml = ps.Model(head)

# Select a recharge model
rch = ps.rch.FlexModel(gw_uptake=True)
# rch = ps.rch.Berendrecht()
# rch = ps.rch.Linear()
# rch = ps.rch.Peterson()

rm = ps.RechargeModel(rain, evap, recharge=rch, rfunc=ps.Gamma(), name="rch")
ml.add_stressmodel(rm)

ml.solve(noise=True, tmin="1990", report="basic")
ml.plots.results(figsize=(10, 6));

## Analyze the estimated recharge flux
After the parameter estimation we can take a look at the recharge flux computed by the model. The flux is easy to obtain using the `get_stress` method of the model object, which automatically provides the optimal parameter values that were just estimated. After this, we can for example look at the yearly recharge flux estimated by the Pastas model.

In [ ]:
recharge = ml.get_stress("rch").resample("A").sum()
ax = recharge.plot.bar(figsize=(10, 3))
ax.set_xticklabels(recharge.index.year)
plt.ylabel("Recharge [mm/year]");

## A few things to keep in mind:
Below are a few things to keep in mind while using the (non-linear) recharge models.

- The use of an appropriate warmup period is necessary, so make sure the precipitation and evaporation are available some time (e.g., one year) before the calibration period.
- Make sure that the units of the precipitation fluxes are in mm/day and that the DatetimeIndex matches exactly.
- It may be possible to fix or vary certain parameters, dependent on the problem. Obtaining better initial parameters may be possible by solving without a noise model first (`ml.solve(noise=False)`) and then solve it again using a noise model.
- For relatively shallow groundwater levels (e.g., < 10 meters), it may be better to use the `Exponential` response function as the the non-linear models already cause a delayed response.

## References
- Berendrecht, W. L., Heemink, A. W., van Geer, F. C., and Gehrels, J. C. (2003), [Decoupling of modeling and measuring interval in groundwater time series analysis based on response characteristics](https://doi.org/10.1016/S0022-1694(03)00075-1), Journal of Hydrology, 278, 1–16.
- Berendrecht, W. L., Heemink, A. W., van Geer, F. C., and Gehrels, J. C. (2006), [A non-linear state space approach to model groundwater fluctuations](https://www.sciencedirect.com/science/article/abs/pii/S0309170805002113), Advances in Water Resources, 29, 959–973.
- Collenteur, R. A., Bakker, M., Klammler, G., and Birk, S. (2021), [Estimation of groundwater recharge from groundwater levels using nonlinear transfer function noise models and comparison to lysimeter data](https://doi.org/10.5194/hess-25-2931-2021), Hydrol. Earth Syst. Sci., 25, 2931–2949.
- Peterson, T.J., and A.W. Western (2014), [Nonlinear time-series modeling of unconfined groundwater head](https://doi.org/10.1002/2013WR014800), Water Resour. Res., 50, 8330-8355.
- Von Asmuth, J.R., Maas, K., Bakker, M. and Petersen, J. (2008), [Modeling Time Series of Ground Water Head Fluctuations Subjected to Multiple Stresses](https://doi.org/10.1111/j.1745-6584.2007.00382.x). Groundwater, 46: 30-40.

## Data Sources
In this notebook we analysed a head time series near the town of De Bilt in the Netherlands. Data is obtained from the following resources:
- The heads (`B32C0639001.csv`) are downloaded from https://www.dinoloket.nl/ 
- The precipitation and evapotranspiration (`rain_260.csv` and `evap_260.csv`) are downloaded from https://knmi.nl